<a href="https://colab.research.google.com/github/JohnMorrisonn/DS-Unit-2-Kaggle-Challenge/blob/master/module4/224_Assignment_kaggle_challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 4

## Catch up, if needed
- [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2/portfolio-project/ds6), then choose your dataset, and [submit this form](https://forms.gle/nyWURUg65x1UTRNV9), due yesterday at 3:59pm Pacific.
- Submit predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file.) The competition closes today at 3:59pm. Every student should make at least one submission that scores at least 60% accuracy (above the majority class baseline).

## Assignment
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your final predictions to our Kaggle competition. Optionally, go to **My Submissions**, and _"you may select up to 1 submission to be used to count towards your final leaderboard score."_ The competition closes today at 3:59pm.
- [ ] Add comments and Markdown to your notebook. Clean up your code.
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Try combining xgboost early stopping, cross-validation, & hyperparameter optimization, with [the "original" (non scikit-learn) xgboost API](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.cv).
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?

#### Try stacking multiple submissions!

Here's some code you can use:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # eli5, version >= 0.9
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders eli5 pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module3')

Requirement already up-to-date: category_encoders in /usr/local/lib/python3.6/dist-packages (2.0.0)
Requirement already up-to-date: eli5 in /usr/local/lib/python3.6/dist-packages (0.9.0)
Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.6/dist-packages (2.3.0)
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.1.0)
Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

In [0]:
def wrangle(X):
    
    '''
    Date-recorded changed to datetime, separated to year, month, and day columns
    Created 'years' column for years from construction to last inspection
    Replaced zero value latitude and logitudes with mean values for the region
    Columns with zeros replaced to NaN values
    Source_type and extraction_type_class grouped into top values + 'Other'
    Created new columns with bins for gps_height and tsh
    Dropped irrelevant columns
    '''
    
    X = X.copy()

    X['year_recorded'] = pd.to_datetime(X['date_recorded']).dt.year
    X['month_recorded'] = pd.to_datetime(X['date_recorded']).dt.month
    X['day_recorded'] = pd.to_datetime(X['date_recorded']).dt.day
    X['years'] = X['year_recorded'] - X['construction_year']

    mask = X['region'] == 'Shinyanga'
    mask_mwanza = X['region'] == 'Mwanza'
    X.loc[mask, 'longitude'] = X.loc[mask, 'longitude'].replace(0, 33.24)
    X.loc[mask_mwanza, 'longitude'] = X.loc[mask_mwanza, 'longitude'].replace(0, 33.09)
    X.loc[mask, 'latitude'] = X.loc[mask, 'latitude'].replace(2e-8, -3.50)
    X.loc[mask_mwanza, 'latitude'] = X.loc[mask_mwanza, 'latitude'].replace(2e-8, -2.62)

    cols_with_zeros = ['construction_year', 'amount_tsh',
                       'population', 'gps_height']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        # X[col+'_MISSING'] = X[col].isnull()
    
    top4 = X['source_type'].value_counts()[:4].index
    X.loc[~X['source_type'].isin(top4), 'source_type'] = 'OTHER'
    top3 = X['extraction_type_class'].value_counts()[:3].index
    X.loc[~X['extraction_type_class'].isin(top3), 'extraction_type_class'] = 'OTHER'

    X['altitude_group'] = pd.qcut(X['gps_height'], 12, labels=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L'])
    X['tsh_new'] = pd.qcut(X['amount_tsh'], 8, labels=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])

    drop_columns = ['id', 'num_private',       
    ]
    X = X.drop(columns=drop_columns)

    return X

train = wrangle(train)
test = wrangle(test) 

In [0]:
target = ['status_group']
features = train.drop(columns=target).columns.tolist()
X_train, X_val, y_train, y_val= train_test_split(train[features], train['status_group'], test_size=0.25, train_size=0.75, stratify=train['status_group'])

In [0]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

In [0]:
pipeline.named_steps['randomforestclassifier']

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators='warn',
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
y_train.shape

(59400,)

In [0]:
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV

target = 'status_group'
features = train.columns.drop(target)
X_train = train[features]
y_train = train[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=42)
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1), 
}

search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3,  
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 13.3min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed: 25.4min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 31.0min finished


In [0]:
print('Best hyperparameters:', search.best_params_)
print('Cross-validation MAE', search.best_score_)

Best hyperparameters: {'randomforestclassifier__max_depth': 20, 'randomforestclassifier__max_features': 0.17069710661411708, 'randomforestclassifier__n_estimators': 70, 'simpleimputer__strategy': 'mean'}
Cross-validation MAE 0.8063468013468014
